In [26]:
# pip install PyQt5
# https://matplotlib.org/3.3.3/tutorials/toolkits/mplot3d.html#toolkit-mplot3d-tutorial
# https://matplotlib.org/3.3.3/gallery/index.html#mplot3d-examples-index
%matplotlib qt
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt


# setup the figure and axes
fig = plt.figure(figsize=(8, 3))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')


# fake data
# _x = np.arange(4) # array([0, 1, 2, 3])
_x = [0, 1, 2, 3]
# _y = np.arange(5) # array([0, 1, 2, 3, 4])
_y = [0, 1, 2, 3,5, 6, 7, 8]
_xx, _yy = np.meshgrid(_x, _y)
x, y = _xx.ravel(), _yy.ravel()
# x # array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])
# y # array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4])
top = x + y # array([0, 1, 2, 3, 1, 2, 3, 4, 2, 3, 4, 5, 3, 4, 5, 6, 4, 5, 6, 7])
bottom = np.zeros_like(top) # array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 
width = depth = 1

ax1.bar3d(x, y, bottom, width, depth, top, shade=True)
ax1.set_title('Shaded')

plt.show()

In [ ]:
# Ok now let's mock up a scenario
# ODA is trading at 10 now, sma30 == 10
# Each months has 3 days (t0, t1, t2)
# Due to exceptionally good quarter and fantastic prospects,
# ODA is expected to skyroket due to good atfer market earnings in t2 of m1

# So we have a build-up of expectations 2 months prior to earnings and 1 month aftermath
# What is option traders' sentiment re fair price: a) currently b) in dynamics?

expirations = ['m0', 'm1', 'm2'] # options expire on t2

# t0
price_t0 = 10
calls_t0 = [
#     [exp, strike, volume, openInterest]
    ['m0', 12, 100, 100],
    ['m1', 14, 100, 100],
    ['m2', 16, 100, 100],
]

# t1
price_t1 = 11 # price increases
calls_t1 = [
#     now o_traders have confirmation bias tell them - see, price goes up,
#     it's less distance to 12, less premium, price might go up higher than
#     we thought yesterday, let's trade a higher strike - hold this thought! 
    
#     we're just gonna stick to our initial strikes now, later will track all strikes and notice movement in o_trader's sentiment
    ['m0', 12, 100, 200], # +100 BTO/STOs 
    ['m1', 14, 100, 200], # +100 BTO/STOs 
    ['m2', 16, 100, 200], # +100 BTO/STOs 
]

# t2 expiration
price_t2 = 12 # price increases
calls_t2 = [
#     first expiration, price hit expected level
    ['m0', 12, 100, 0], # +100 BTC/STC - half of openInterest was liquidated, half expired/assigned  
    ['m1', 14, 100, 300], # +100 BTO/STOs 
    ['m2', 16, 100, 300], # +100 BTO/STOs
]

# t3
price_t3 = 12 # price remains level
calls_t3 = [
#     first signs of hesitation
    ['m0', 12, 0, 0],
    ['m1', 14, 100, 350], # +50 BTO/STOs -50 BTC/STC - some dropped out
    ['m2', 16, 100, 400], # +100 BTO/STOs
]

# t4
price_t4 = 11.5 # price drops
calls_t4 = [
#     wops, sellers get slimmer premiums coz it's les likely (risky) that call expires ITM
    ['m0', 12, 0, 0],
    ['m1', 14, 100, 400], # +50 BTO/STOs -50 BTC/STC - some dropped out
    ['m2', 16, 50, 425], # +25 BTO/STOs -25 BTC/STC - some dropped out
]

# t5 expiration
price_t5 = 14.5 # price skyrokets as everyone catches up with how awesome ODA is
calls_t5 = [
#     
    ['m0', 12, 0, 0],
    ['m1', 14, 400, 0], # -400 BTC/STC - everybody liquidated realizing that ODA is gonna go up fast
    ['m2', 16, 200, 625], # +200 BTO/STOs
]

# t6 
price_t6 = 15 # price skyrokets due to fantastic earnings
calls_t6 = [
#     
    ['m0', 12, 0, 0],
    ['m1', 14, 400, 0], # -400 BTC/STC - everybody liquidated realizing that ODA is gonna go up fast
    ['m2', 16, 200, 625], # +200 BTO/STOs
]

# t7 
price_t7 = 16 # price continues to rise
calls_t7 = [
#     
    ['m0', 12, 0, 0],
    ['m1', 14, 0, 0],
    ['m2', 16, 300, 725], # +100 BTO/STOs -200 BTC/STC - some bought ATM some bailed out
]

# t8 expiration
price_t8 = 20 # sky is the limit
calls_t8 = [
#     
    ['m0', 12, 0, 0],
    ['m1', 14, 0, 0],
    ['m2', 16, 300, 0], # +100 BTO/STOs -200 BTC/STC - some bought ATM some bailed out
]

# OK after doing this I see that there are 2 3d plots here:
# a) the above scenario tells us WHAT'S CURRENT SENTIMENT AS PER STRIKE? - ie focus on FIXED strike, we zoom in on chosen strike
#    since [exp, strike] are FIXED, we should combine them as x axis (ie contractSymbol) and plot openInterest on z (with volumes
#    represented by colors - the redder - the more BTC&STC / BTO&STO)
# b) we would like to find out WHAT'S THE MOST TRADED STRIKE? - ie we want to know what o_traders think of future und price
# now arrange x, y, z
exps = []

